In [8]:
import pandas as pd
import numpy as np
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Load CSV files
train_data = pd.read_csv("C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data/train_images.csv")
test_data = pd.read_csv("C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data/test_images_sample.csv")
class_names = np.load("C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data/class_names.npy", allow_pickle=True).item()

# Custom Dataset class
class BirdDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        img_path = "./images/"+ self.data.iloc[idx, 0]  # Image file path
        image = Image.open(img_path).convert("RGB")
        label = self.data.iloc[idx, 1] - 1  # Adjust labels to be 0-indexed
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = BirdDataset(csv_file="C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data/train_images.csv", img_dir="train_images", transform=transform)
test_dataset = BirdDataset(csv_file="C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data/test_images_sample.csv", img_dir="test_images", transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [9]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Modify the final layer for 200 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\Gebruiker\anaconda3\envs\AML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gebruiker\anaconda3\envs\AML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
import torch.optim as optim

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
print("Current working directory:", os.getcwd())
# Change the working directory to the folder containing the files
os.chdir("C:/Users/Gebruiker/Documents/UVA/Vakken met code/AML/AML2024-Feather-in-Focus/data")
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\Gebruiker\Documents\UVA\Vakken met code\AML\AML2024-Feather-in-Focus
Current working directory: C:\Users\Gebruiker\Documents\UVA\Vakken met code\AML\AML2024-Feather-in-Focus\data


In [13]:
def train_model(model, dataloader, criterion, optimizer, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss/len(dataloader)}")

train_model(model, train_loader, criterion, optimizer, device)


Epoch 1/10, Loss: 5.325649528968625


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
    return predictions


In [ ]:
test_predictions = evaluate_model(model, test_loader, device)

# Prepare submission
test_data['label'] = test_predictions
test_data.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")
